In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # Import train_test_split function
import warnings
warnings.filterwarnings("ignore")
import os

In [2]:
import optuna
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import joblib

In [2]:
X_train_balanced = pd.read_csv(r'C:\Users\sastr\6th sem\Internship\NSL_KDD\x_train_balanced.csv')

In [3]:
X_train_balanced.head()

,src_bytes,flag_9,count,diff_srv_rate,dst_host_same_srv_rate,same_srv_rate,dst_bytes,serror_rate,dst_host_srv_count,dst_host_diff_srv_rate,...,dst_host_srv_diff_host_rate,protocol_type_0,service_24,service_14,service_49,protocol_type_1,service_15,srv_diff_host_rate,rerror_rate,wrong_fragment
0,-0.006410,0.825150,-0.647181,-0.349683,-0.737818,0.771283,-0.004919,-0.637209,-0.538860,-0.227350,...,0.066252,-0.265429,-0.686328,-0.194371,-0.458130,0.476175,-0.158232,-0.374560,-0.374362,-0.089486
1,-0.007755,0.825150,-0.708312,-0.349683,0.888207,0.771283,-0.004908,-0.637209,1.069056,-0.386146,...,-0.289103,-0.265429,-0.686328,-0.194371,-0.458130,-2.100067,-0.158232,0.895504,-0.374362,-0.089486
2,-0.007762,-1.211901,-0.725778,-0.349683,0.955030,0.771283,-0.004919,-0.637209,1.141322,-0.386146,...,-0.289103,-0.265429,1.457030,-0.194371,-0.458130,0.476175,-0.158232,-0.374560,2.746403,-0.089486
3,-0.007725,0.825150,-0.629715,-0.349683,1.066401,0.771283,-0.004595,-0.637209,1.258754,-0.439078,...,0.155091,-0.265429,1.457030,-0.194371,-0.458130,0.476175,-0.158232,-0.374560,-0.374362,-0.089486
4,-0.007762,-1.211901,0.846162,-0.016930,-1.116482,-1.435162,-0.004919,-0.637209,-0.999555,-0.068553,...,-0.289103,-0.265429,-0.686328,-0.194371,2.182788,0.476175,-0.158232,-0.374560,2.746403,-0.089486


In [3]:
X_test_selected = pd.read_csv(r'C:\Users\sastr\6th sem\Internship\NSL_KDD\x_test_selected.csv')

In [5]:
X_test_selected.head()

,src_bytes,flag_9,count,diff_srv_rate,dst_host_same_srv_rate,same_srv_rate,dst_bytes,serror_rate,dst_host_srv_count,dst_host_diff_srv_rate,...,dst_host_srv_diff_host_rate,protocol_type_0,service_24,service_14,service_49,protocol_type_1,service_15,srv_diff_host_rate,rerror_rate,wrong_fragment
0,-0.007348,0.825150,-0.725778,-0.349683,-0.916013,0.771283,-0.004801,-0.637209,-0.529826,-0.333214,...,0.066252,-0.265429,-0.686328,-0.194371,-0.45813,0.476175,-0.158232,-0.374560,-0.374362,-0.089486
1,-0.007755,0.825150,0.942225,-0.294224,0.620915,0.748536,-0.004885,-0.637209,0.355431,-0.333214,...,-0.289103,-0.265429,-0.686328,-0.194371,-0.45813,-2.100067,-0.158232,-0.336073,-0.374362,-0.089486
2,-0.007755,0.825150,0.322182,-0.349683,1.066401,0.771283,-0.004899,-0.637209,1.258754,-0.439078,...,-0.289103,-0.265429,-0.686328,-0.194371,-0.45813,-2.100067,-0.158232,-0.374560,-0.374362,-0.089486
3,-0.007738,0.825150,-0.717045,-0.349683,-1.138756,0.771283,-0.004893,-0.637209,-1.026654,4.007224,...,-0.289103,-0.265429,-0.686328,-0.194371,-0.45813,-2.100067,-0.158232,-0.374560,-0.374362,-0.089486
4,-0.007762,-1.211901,-0.717045,-0.349683,-1.138756,0.771283,-0.004919,-0.637209,-1.026654,3.160309,...,-0.289103,-0.265429,-0.686328,-0.194371,-0.45813,0.476175,-0.158232,-0.374560,2.746403,-0.089486


In [6]:
y_train_balanced = pd.read_csv(r'C:\Users\sastr\6th sem\Internship\NSL_KDD\y_train_balanced.csv')

In [7]:
y_train_balanced.head()

,labels
0,0
1,0
2,0
3,0
4,1


In [4]:
y_test = pd.read_csv(r'C:\Users\sastr\6th sem\Internship\NSL_KDD\y_test.csv')

In [9]:
y_test.head()

,labels
0,0
1,0
2,0
3,0
4,2


**Train validation split**

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X_train_balanced, y_train_balanced, test_size=0.2, random_state=42)

# Hyperparameter tuning

In [17]:
def objective(trial):
    # Suggest hyperparameters
    param = {
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': 5,
        'booster': 'gbtree',
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'eta': trial.suggest_loguniform('eta', 1e-3, 0.3),
        'gamma': trial.suggest_loguniform('gamma', 1e-3, 10.0),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 9),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000)
    }

    model = xgb.XGBClassifier(**param, use_label_encoder=False)
    model.fit(X_train, y_train)

    # Predict on the validation set
    y_pred_val = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred_val)
    return accuracy


In [18]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

[I 2024-06-28 15:43:51,295] A new study created in memory with name: no-name-72b691c2-6052-43d6-bbce-1fdda8ea3845
[I 2024-06-28 15:43:54,419] Trial 0 finished with value: 0.997879442371883 and parameters: {'lambda': 1.8780007913090637, 'alpha': 0.003583430095201186, 'eta': 0.07428290902502366, 'gamma': 0.004295149370646253, 'max_depth': 8, 'min_child_weight': 2, 'subsample': 0.9610880965974227, 'colsample_bytree': 0.546243049454192, 'n_estimators': 111}. Best is trial 0 with value: 0.997879442371883.
[I 2024-06-28 15:44:07,038] Trial 1 finished with value: 0.9979579815432947 and parameters: {'lambda': 0.012169381402880996, 'alpha': 0.06728402804294017, 'eta': 0.06365856867059035, 'gamma': 0.7430481687380299, 'max_depth': 8, 'min_child_weight': 1, 'subsample': 0.6683772572017297, 'colsample_bytree': 0.6517698777015299, 'n_estimators': 765}. Best is trial 1 with value: 0.9979579815432947.
[I 2024-06-28 15:44:21,312] Trial 2 finished with value: 0.996347928529354 and parameters: {'lambda'

In [19]:
print("Best hyperparameters: ", study.best_params)

Best hyperparameters:  {'lambda': 2.595583407048943, 'alpha': 0.022492558225967765, 'eta': 0.1188019426033616, 'gamma': 0.0022824270480267927, 'max_depth': 9, 'min_child_weight': 8, 'subsample': 0.6791034151223656, 'colsample_bytree': 0.5493911544882935, 'n_estimators': 889}


In [21]:
best_params = study.best_params
best_model = xgb.XGBClassifier(**best_params, use_label_encoder=False)
best_model.fit(X_train, y_train)

XGBClassifier(alpha=0.022492558225967765, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5493911544882935, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eta=0.1188019426033616, eval_metric=None, feature_types=None,
              gamma=0.0022824270480267927, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              lambda=2.595583407048943, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=8, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=889, ...)

In [22]:
y_pred_val = best_model.predict(X_val)

In [23]:
accuracy = accuracy_score(y_val, y_pred_val)
precision = precision_score(y_val, y_pred_val, average='weighted')
recall = recall_score(y_val, y_pred_val, average='weighted')
f1 = f1_score(y_val, y_pred_val, average='weighted')

print("Validation Accuracy:", accuracy)
print("Validation Precision:", precision)
print("Validation Recall:", recall)
print("Validation F1-score:", f1)

Validation Accuracy: 0.9985862949145886
Validation Precision: 0.9985623473182709
Validation Recall: 0.9985862949145886
Validation F1-score: 0.9985550285637929


In [24]:
print("\nClassification Report:")
print(classification_report(y_val, y_pred_val))


Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13221
           1       1.00      1.00      1.00      9314
           2       1.00      1.00      1.00      2575
           3       0.98      0.97      0.97       341
           4       0.89      0.57      0.70        14

    accuracy                           1.00     25465
   macro avg       0.97      0.91      0.93     25465
weighted avg       1.00      1.00      1.00     25465



In [27]:
y_pred_test = best_model.predict(X_test_selected)

# Calculate classification metrics
accuracy = accuracy_score(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test, average='weighted')
recall = recall_score(y_test, y_pred_test, average='weighted')
f1 = f1_score(y_test, y_pred_test, average='weighted')

print("Test Accuracy:", accuracy)
print("Test Precision:", precision)
print("Test Recall:", recall)
print("Test F1-score:", f1)

# Generate and print the classification report
print("\nClassification Report for Test Set:")
print(classification_report(y_test, y_pred_test))

Test Accuracy: 0.9314673527324343
Test Precision: 0.9361080906035102
Test Recall: 0.9314673527324343
Test F1-score: 0.9191649574652552

Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.89      0.99      0.94     11245
           1       0.99      0.96      0.97      8095
           2       0.97      0.90      0.93      2157
           3       0.97      0.16      0.28      1009
           4       0.92      0.32      0.47        38

    accuracy                           0.93     22544
   macro avg       0.95      0.67      0.72     22544
weighted avg       0.94      0.93      0.92     22544



In [28]:
joblib.dump(best_model, 'xgboost_best_model.pkl')

['xgboost_best_model.pkl']

**Generate synthetic samples**

In [30]:
def generate_synthetic_samples(data, num_samples=5, noise_level=0.1):
    synthetic_samples = []
    for _ in range(num_samples):
        sample = data.sample(1).values.flatten()
        # Add random noise to create a synthetic sample
        noise = np.random.normal(0, noise_level, sample.shape)
        synthetic_sample = sample + noise
        synthetic_samples.append(synthetic_sample)
    return np.array(synthetic_samples)

# Generate synthetic samples
num_synthetic_samples = 5
synthetic_samples = generate_synthetic_samples(X_test_selected, num_samples=num_synthetic_samples)
print(synthetic_samples)
# Convert to DataFrame
#synthetic_samples_df = pd.DataFrame(synthetic_samples, columns=x_test_selected.columns)

[[-0.17661139  1.01918539 -0.61515032 -0.17634577  1.07563434  0.92494331
  -0.01421722 -0.7196347   1.2324522  -0.3409582  -0.80457796 -0.04960275
   1.36565228  0.13036489 -0.03342085 -0.27987732  1.41022895 -0.14086903
  -0.29959668  0.4703312  -0.07524982 -0.26123093 -0.35380755 -0.05003695]
 [-0.0820833   0.84738056 -0.68179348 -0.29284299  1.10564479  0.66514164
  -0.07673382 -0.68371236  1.37092075 -0.4751195  -0.46513566 -0.28015819
   1.22654529  0.81805586 -0.47685809 -0.27650542  1.43666888 -0.00763187
  -0.46620975  0.34201512 -0.14781268 -0.18685975 -0.28903285 -0.23686792]
 [-0.08895864 -1.17620755 -0.73902883  5.38928375 -1.24529336 -0.35994855
   0.10695013 -0.57464589 -0.99976155  1.70429673  0.59025225 -0.49899241
  -0.95230114  0.60016878 -0.42011446 -0.40734016 -0.72467558 -0.20582802
   2.16719992  0.54855502 -0.16603608 -0.23689432  2.74400844 -0.05647474]
 [-0.01617648  0.78113787 -0.72272377 -0.38238918  1.15599369  1.01678488
   0.15032637 -0.46898106  0.837241

In [8]:
import numpy as np

# Convert y_test to a numpy array if it's not already
y_test = np.array(y_test)

# Initialize a dictionary to store the indices of each class label
class_indices = {}

# Loop through the unique class labels and find the first occurrence index
for class_label in np.unique(y_test):
    index = np.where(y_test == class_label)[0][0]
    class_indices[class_label] = index

# Extract rows for each class label from X_test_selected
rows_for_classes = {label: X_test_selected.iloc[index].tolist() for label, index in class_indices.items()}

# Print the extracted rows
for label, row in rows_for_classes.items():
    print(f"Row for class {label}:")
    print(row)
    print()


Row for class 0:
[-0.0073484634653608, 0.8251500711603638, -0.7257784945454897, -0.349683030873482, -0.9160127180365886, 0.7712831058493207, -0.0048005220030914, -0.6372092679572258, -0.5298264288361667, -0.3332138401374709, -0.4478339589698615, -0.3681102129968653, 1.2356940323701655, -0.0216614774298762, 0.0662520251098218, -0.2654290405834508, -0.6863277040077078, -0.1943707614720423, -0.4581297059858955, 0.4761752658981646, -0.1582322028606456, -0.3745597044055346, -0.3743622399196752, -0.089486422020401]

Row for class 1:
[-0.0077622407405687, -1.2119007619956388, 1.081953049767606, -0.0169295977079487, -1.0719329252940557, -1.4124154513771747, -0.0049186443837248, 1.602663889932865, -0.963421750682496, -0.068553020255113, -0.4801968475158174, -0.2579713092287289, -0.8092618187059747, 0.7343425609306344, -0.2891034002628785, -0.2654290405834508, -0.6863277040077078, -0.1943707614720423, 2.182787946151624, 0.4761752658981646, -0.1582322028606456, -0.3745597044055346, -0.37436223991

In [7]:
rows_for_classes

{0: src_bytes                     -0.007348
 flag_9                         0.825150
 count                         -0.725778
 diff_srv_rate                 -0.349683
 dst_host_same_srv_rate        -0.916013
 same_srv_rate                  0.771283
 dst_bytes                     -0.004801
 serror_rate                   -0.637209
 dst_host_srv_count            -0.529826
 dst_host_diff_srv_rate        -0.333214
 dst_host_same_src_port_rate   -0.447834
 srv_count                     -0.368110
 logged_in                      1.235694
 dst_host_count                -0.021661
 dst_host_srv_diff_host_rate    0.066252
 protocol_type_0               -0.265429
 service_24                    -0.686328
 service_14                    -0.194371
 service_49                    -0.458130
 protocol_type_1                0.476175
 service_15                    -0.158232
 srv_diff_host_rate            -0.374560
 rerror_rate                   -0.374362
 wrong_fragment                -0.089486
 Name: 0, dty